In [1]:
# print information
print('\n')
print('SynthSR prediction')
print('\n')

# python imports
import os
import sys
import numpy as np
from argparse import ArgumentParser



SynthSR prediction




In [2]:
# !export PATH=/home/viscent/anaconda3/envs/synthsr/bin:$PATH
# !export LD_LIBRARY_PATH=/home/viscent/anaconda3/envs/synthsr/lib:$LD_LIBRARY_PATH
!export PATH=/usr/local/cuda-10.0/bin:$PATH
!export LD_LIBRARY_PATH=/usr/local/cuda-10.0/lib64:$LD_LIBRARY_PATH
!export CUDA_VISIBLE_DEVICES=-1

### add main folder to python path and import SynthSR packages

In [3]:
synthSR_home = '/home/viscent/hdd/viscent/SynthSR'
print(synthSR_home)
sys.path.append(synthSR_home)
from ext.neuron import models as nrn_models
from ext.lab2im import utils
from ext.lab2im import edit_volumes
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

/home/viscent/hdd/viscent/SynthSR


Using TensorFlow backend.


In [4]:
unet_model = nrn_models.unet(nb_features=24,
                             input_shape=[None, None, None, 1],
                             nb_levels=5,
                             conv_size=3,
                             nb_labels=1,
                             feat_mult=2,
                             nb_conv_per_level=2,
                             conv_dropout=0,
                             final_pred_activation='linear',
                             batch_norm=-1,
                             activation='elu',
                             input_model=None)


Building enc...


2022-04-12 17:12:04.165480: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-04-12 17:12:04.187551: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-04-12 17:12:04.187573: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: fv
2022-04-12 17:12:04.187576: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: fv
2022-04-12 17:12:04.187627: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-04-12 17:12:04.187640: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2022-04-12 17:12:04.187643: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
2022-04-12 17:12:04.187815: I tensorflow/core/platform/cpu_feature_guard.cc:14

Building dec...
dec built


In [5]:
unet_model.load_weights(os.path.join(synthSR_home, 'models/SynthSR_v10_210712.h5'), by_name=True)

#### Prepare list of images to process

In [6]:
path_images = '/media/hdd/viscent/dHCP/synthsr_data/images_t1_skull_1mm'
basename = os.path.basename(path_images)
path_predictions = '/media/hdd/viscent/SynthSR/results'

In [7]:
if ('.nii.gz' not in basename) & ('.nii' not in basename) & ('.mgz' not in basename) & ('.npz' not in basename):
    if os.path.isfile(path_images):
        raise Exception('extension not supported for %s, only use: nii.gz, .nii, .mgz, or .npz' % path_images)
    images_to_segment = utils.list_images_in_folder(path_images)
    utils.mkdir(path_predictions)
    path_predictions = [os.path.join(path_predictions, os.path.basename(image)).replace('.nii', '_SynthSR.nii') for image in
                   images_to_segment]
    path_predictions = [seg_path.replace('.mgz', '_SynthSR.mgz') for seg_path in path_predictions]
    path_predictions = [seg_path.replace('.npz', '_SynthSR.npz') for seg_path in path_predictions]

else:
    assert os.path.isfile(path_images), "files does not exist: %s " \
                                        "\nplease make sure the path and the extension are correct" % path_images
    images_to_segment = [path_images]
    path_predictions = [path_predictions]


In [8]:
print('Found %d images' % len(images_to_segment))
for idx, (path_image, path_prediction) in enumerate(zip(images_to_segment, path_predictions)):
    print('  Working on image %d ' % (idx+1))
    print('  ' + path_image)

    im, aff, hdr = utils.load_volume(path_image,im_only=False,dtype='float')
    im, aff = edit_volumes.resample_volume(im, aff, [1.0, 1.0, 1.0])
    aff_ref = np.eye(4)
    im, aff2 = edit_volumes.align_volume_to_ref(im, aff, aff_ref=aff_ref, return_aff=True, n_dims=3)
    im = im - np.min(im)
    im = im / np.max(im)
    I = im[np.newaxis,..., np.newaxis]
    W = (np.ceil(np.array(I.shape[1:-1]) / 32.0) * 32).astype('int')
    idx = np.floor((W-I.shape[1:-1])/2).astype('int')
    S = np.zeros([1, *W, 1])
    S[0, idx[0]:idx[0]+I.shape[1], idx[1]:idx[1]+I.shape[2], idx[2]:idx[2]+I.shape[3], :] = I
    output = unet_model.predict(S)
    pred = np.squeeze(output)
    pred = 255 * pred
    pred[pred<0] = 0
    pred[pred>128] = 128
    pred = pred[idx[0]:idx[0]+I.shape[1], idx[1]:idx[1]+I.shape[2], idx[2]:idx[2]+I.shape[3]]
    utils.save_volume(pred,aff2,None,path_prediction)

print(' ')
print('All done!')
print(' ')

Found 4 images
  Working on image 1 
  /media/hdd/viscent/dHCP/synthsr_data/images_t1_skull_1mm/sub-CC00051XX02_ses-7702_T1w_1mm.nii.gz
  Working on image 2 
  /media/hdd/viscent/dHCP/synthsr_data/images_t1_skull_1mm/sub-CC00052XX03_ses-8300_T1w_1mm.nii.gz
  Working on image 3 
  /media/hdd/viscent/dHCP/synthsr_data/images_t1_skull_1mm/sub-CC00053XX04_ses-8607_T1w_1mm.nii.gz
  Working on image 4 
  /media/hdd/viscent/dHCP/synthsr_data/images_t1_skull_1mm/sub-CC00054XX05_ses-8800_T1w_1mm.nii.gz
 
All done!
 


In [9]:
import SimpleITK as sitk

%matplotlib notebook
import matplotlib.pyplot as plt
input = os.path.join(path_images,'sub-CC00051XX02_ses-7702_T1w_brain_1mm.nii.gz')
output = os.path.join('/media/hdd/viscent/SynthSR/results','sub-CC00051XX02_ses-7702_T1w_brain_1mm_SynthSR.nii.gz')

ModuleNotFoundError: No module named 'SimpleITK'